In [2]:
from embed_software.preprocess import claims_processor, readme_processor, Preprocessor, ParallelProcessor       

In [ ]:
import s3fs

inpath = 'oecd-scraping/indeed-us'
string_processor = Preprocessor(readme_processor, 4).process
p = ParallelProcessor(string_processor, inpath, 'us-jobs-lookup.db', 'description', 'url', cores=6, fs = s3fs.S3FileSystem())
p.process_all()